In [1]:
!pip install pickle-mixin
!pip install markovify

  Preparing metadata (setup.py) ... done
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5991 sha256=bfc156bd1ad1d0b8ac01ac59b75270f8cf298166e4807c6ffd34fc977b7f8693
  Stored in directory: /root/.cache/pip/wheels/3e/c6/e9/d1b0a34e1efc6c3ec9c086623972c6de6317faddb2af0a619c
Successfully built pickle-mixin
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 4.9 MB/s eta 0:00:00
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18607 sha256=5b67f0e60a04c2850fdeef9541308b84dba711268db046e49e4c347a09c812b6
  Stored in directory: /root/.cache/pip/wheels/ca/8c/c5/41413e24c484f883a100c63ca7b3b0362b7c6f6eb6d7c9cc7f
Successfully built markovify


In [37]:
import pandas as pd
from tqdm import tqdm
df = pd.read_csv("/content/Arabic_poetry_dataset.csv", encoding="utf-8")
allowed_chars = ['ض', 'ص', 'ث', 'ق', 'ف', 'غ', 'ع', 'ه', 'خ', 'ح', 'ج', 'د', 'ش', 'س', 'ي', 'ب', 'ل', 'ا', 'ت', 'ن', 'م',
                 'ك', 'ط', 'ئ', 'ء', 'ؤ', 'ر' , 'ى', 'ة', 'و', 'ز', 'ظ', 'إ' ,'أ', 'آ', ' ', '\n', 'ذ', 'ّ']

def delete_puncs(poem):
    for l in poem:
        if l not in allowed_chars:
            poem = poem.replace(l, "")
    return poem

import os

def extract_poems(poet_name):
    output_directory = '/content/input/'
    os.makedirs(output_directory, exist_ok=True)

    with open(os.path.join(output_directory, '{}.txt'.format(poet_name)), 'w', encoding="utf-8") as f:
        for i in range(len(df)):
            if df['poet_name'][i] == poet_name:
                poem = delete_puncs(df['poem_text'][i])
                f.write("{}\n\n".format(poem))


In [38]:
all_poets = set()
for poet_name in df['poet_name']:
    all_poets.add(poet_name)

In [39]:
extract_poems('نزار قباني')
extract_poems('محمد رسول الله')

In [40]:
poet_names_in_dataset = df['poet_name'].unique()
print(poet_names_in_dataset)

['خلفان بن مصبح' 'سالم بن عبدالله الكراني' 'حماد بن سعيد'
 'مبارك بن حمد العقيلي' 'علي بن محمد الشحي' 'محمد بن حمود الشحي'
 'محمد بن صالح المنتفقي' 'يوسف بن رويسم' 'هدى السعدي' 'مانع سعيد العتيبة'
 'عارف الخاجة' 'حمد بن خليفة أبو شهاب' 'كريم معتوق'
 'سالم أبو جمهور القبيسي' 'قاسم حداد' 'أحلام مستغانمي' 'سهام آل براهمي'
 'جلواح' 'عبد القادر الجزائري' 'سليمان بن سحمان' 'عبد الرحمن بن مساعد'
 'أحمد اللهيب' 'عبدالرحمن العشماوي' 'شريف بقنه' 'ابن بشير الإحسائي'
 'سامي المالكي' 'غازي القصيبي' 'محمد بن عبود العمودي'
 'مدثر بن إبراهيم بن الحجاز' 'محمد عبد الباري' 'إدريس جمّاع'
 'حمزة الملك طمبل' 'احمد مطر' 'نازك الملائكة' 'بدر شاكر السياب'
 'محمد مهدي الجواهري' 'عبد الرزاق عبد الواحد' 'بهاء الدين الصيادي'
 'معروف الرصافي' 'أبو الفيض الكتاني' 'مَحمد اسموني' 'شاعر الحمراء'
 'عبدالله البردوني' 'عبد الولي الشميرى' 'ابن شهاب العلوي' 'محمد الشوكاني'
 'ابن طاهر' 'ابن رشيق القيرواني' 'الشاذلي خزنه دار' 'أبو القاسم الشابي'
 'سليم عبدالقادر' 'نزار قباني' 'خليل مردم بك' 'بطرس كرامة'
 'أبو الهدى الصيادي' '

In [41]:
import pickle

with open('/content/vocabs.pkl', 'rb') as pickle_load:
    voc_list = pickle.load(pickle_load)


allowed_chars = ['ذ', 'ض', 'ص', 'ث', 'ق', 'ف', 'غ', 'ع', 'ه', 'خ', 'ح', 'ج', 'د',
                 'ش', 'س', 'ي', 'ب', 'ل', 'ا', 'أ', 'ت', 'ن', 'م', 'ك', 'ط', 'ئ', 'ء', 'ؤ', 'ر', 'ى',
                 'ة', 'و', 'ز', 'ظ', 'ّ', ' ']

max_word_length = 9

In [42]:
#extract all words on the same rythmes

def rhymes_with(word):
    if word not in ['الله', 'والله', 'بالله', 'لله', 'تالله']:
        word = word.replace('ّ', '')
    ending = word[-2:]
    rhymes = []
    for w in voc_list:
        if len(w) < max_word_length and w.endswith(ending):
            rhymes.append(w)
    return rhymes

def rhymes_with_last_n_chars(word, n):
    if word not in ['الله', 'والله', 'بالله', 'لله', 'تالله', 'فالله']:
        word = word.replace('ّ', '')
    ending = word[-n:]
    rhymes = []
    for w in voc_list:
        if len(w) < max_word_length and w.endswith(ending):
            rhymes.append(w)
    return rhymes

In [43]:
import markovify
from tqdm import tqdm

def markov(text_file):
    with open(text_file, 'r', encoding='utf-8') as f:
        text = f.read()
    text_model = markovify.NewlineText(text)
    return text_model

In [44]:
def generate_poem_single_rhyme(poet_name, rhyme, iterations=3000, use_tqdm=False):
    n_of_rhyme_letters = len(rhyme)
    input_file = '/content/input/{}.txt'.format(poet_name)
    text_model = markov(input_file)
    rhymes_list = rhymes_with_last_n_chars(rhyme, n_of_rhyme_letters)
    bayts = set()
    used_rhymes = set()

    poem = ""

    if use_tqdm == True:
        if hasattr(tqdm, '_instances'): tqdm._instances.clear()
        it_range = tqdm(range(iterations))
    else:
        it_range = range(iterations)

    for i in it_range:
        bayt = text_model.make_short_sentence(280, tries=100)
        last_word = bayt.split()[-1]
        if (last_word in rhymes_list) and (last_word not in used_rhymes) and (bayt not in bayts):
            bayts.add(bayt)
            used_rhymes.add(last_word)
            poem += "{}\n".format(bayt)
            if not use_tqdm:
                print(bayt)
    return poem

def generate_poem_2_rhymes(poet_name, rhyme_1, rhyme_2, iterations=3000, use_tqdm=False):
    n_of_rhyme_1_letters = len(rhyme_1)
    n_of_rhyme_2_letters = len(rhyme_2)

    input_file = '/content/input/{}.txt'.format(poet_name)
    text_model = markov(input_file)

    rhymes_1_list = rhymes_with_last_n_chars(rhyme_1, n_of_rhyme_1_letters)
    rhymes_2_list = rhymes_with_last_n_chars(rhyme_2, n_of_rhyme_2_letters)

    bayts_1 = set()
    bayts_2 = set()

    used_rhymes_1 = set()
    used_rhymes_2 = set()

    poem = ""

    if use_tqdm == True:
        if hasattr(tqdm, '_instances'): tqdm._instances.clear()
        it_range = tqdm(range(iterations))
    else:
        it_range = range(iterations)

    for i in it_range:
        bayt = text_model.make_short_sentence(280, tries=100)
        last_word = bayt.split()[-1]

        if (last_word in rhymes_1_list) and (last_word not in used_rhymes_1) and (bayt not in bayts_1):
            bayts_1.add(bayt)
            used_rhymes_1.add(last_word)

        if (last_word in rhymes_2_list) and (last_word not in used_rhymes_2) and (bayt not in bayts_2):
            bayts_2.add(bayt)
            used_rhymes_2.add(last_word)

    len_of_poem = min(len(bayts_1), len(bayts_2))
    for i in range(len_of_poem):
        poem += "{}\n{}\n".format(list(bayts_1)[i], list(bayts_2)[i])

    return poem

In [45]:
x = generate_poem_single_rhyme(
                            'نزار قباني',
                            'ايا',
                            iterations=3000,
                            use_tqdm=False
                            )

ورأسا بين آلاف الهدايا
كيف الدخول إلى القصيدة يا ترى اخترع القصيدة والنبيذ وخصر مايا
يا بيتها في آخر الليل مثل الشظايا
و في كل الزوايا
